In [14]:
from lib.imports import *
import os

In [15]:
normalize = True
balance = False
centered = True

window_size = 501
stride = 1 # in datapoints
block_size = 1

train_size = .5
dev_size = .25
test_size = .25
n_files = 20
n_holdout_files = 1

files = list(range(71))
skip_idx = [19,24,26,32,34,38,40,45,52,55,70]
for idx in skip_idx:
    files.remove(idx)
fileindices = random.sample(files,n_files)
holdout_indices = fileindices[(n_files-n_holdout_files):]
print(fileindices,holdout_indices)

[1, 14, 43, 15, 16, 42, 58, 44, 2, 39, 5, 21, 61, 17, 30, 4, 64, 46, 22, 6] [6]


In [16]:
from lib.utils import load_nursing_by_index

In [12]:
X,y = load_nursing_by_index(0)
torch.bincount(y.long())
# import plotly.express as px
# df = pd.DataFrame([X[:,0].numpy(),y.argmax(axis=1).numpy()]).T
# fig = px.line(df)
# fig.show(renderer='browser')

tensor([138679,   4321])

In [17]:
y_train_all = torch.Tensor()
y_dev_all = torch.Tensor()
y_test_all = torch.Tensor()
x_train_i = 0
x_dev_i = 0
x_test_i = 0
data_dir = f'w{window_size}_centered_tiny_x'
os.makedirs(data_dir)
os.makedirs(f'{data_dir}/train')
os.makedirs(f'{data_dir}/dev')
os.makedirs(f'{data_dir}/test')
config = {
    'TRAIN_SIZE':train_size,
    'DEV_SIZE':dev_size,
    'TEST_SIZE':test_size,
    'FILES':fileindices,
    'NORMALIZED':normalize,
    'BALANCED':balance,
    'HOLDOUT_FILES':holdout_indices
}
with open(f'{data_dir}/config.json', 'w') as f:
     f.write(json.dumps(config))

In [18]:
for fileindex in fileindices:
    X,y_true = load_nursing_by_index(fileindex)

    if (fileindex in holdout_indices):
        torch.save((X,y_true),f'{data_dir}/holdout_{fileindex}.pt')
        continue
    
    x = X[:,0].unsqueeze(1)
    xs = [x[:-(window_size-1)]]
    for i in range(1,window_size-1):
        xs.append(x[i:i-(window_size-1)])
    xs.append(x[(window_size-1):])
    X = torch.cat(xs,axis=1).float()
    y_true = y_true[window_size//2:-(window_size//2)].unsqueeze(1)

    # train test split
    X_train,X_test,y_train,y_test = train_test_split(X,y_true,test_size=(1-train_size),shuffle=True,stratify=y_true,random_state=0)
    X_dev,X_test,y_dev,y_test = train_test_split(X_test,y_test,test_size=(test_size/(1-train_size)),shuffle=True,stratify=y_test,random_state=0)

    y_train_all = torch.cat([y_train_all,y_train])
    y_dev_all = torch.cat([y_dev_all,y_dev])
    y_test_all = torch.cat([y_test_all,y_test])

    for Xi in X_train:
        torch.save(Xi.clone(),f'{data_dir}/train/{x_train_i}.pt')
        x_train_i += 1
    for Xi in X_dev:
        torch.save(Xi.clone(),f'{data_dir}/dev/{x_dev_i}.pt')
        x_dev_i += 1
    for Xi in X_test:
        torch.save(Xi.clone(),f'{data_dir}/test/{x_test_i}.pt')
        x_test_i += 1
torch.save(y_train_all,f'{data_dir}/y_train.pt')
torch.save(y_dev_all,f'{data_dir}/y_dev.pt')
torch.save(y_test_all,f'{data_dir}/y_test.pt')

In [19]:
# test if worked
from lib.datasets import Dataset2p0
from torch.utils.data import DataLoader
trainloader = DataLoader(Dataset2p0(dir=f'{data_dir}/train/',labels=f'{data_dir}/y_train.pt'),batch_size=1,shuffle=True)
devloader = DataLoader(Dataset2p0(dir=f'{data_dir}/dev/',labels=f'{data_dir}/y_test.pt'),batch_size=1,shuffle=True)
testloader = DataLoader(Dataset2p0(dir=f'{data_dir}/test/',labels=f'{data_dir}/y_test.pt'),batch_size=1,shuffle=True)
X,y = next(iter(trainloader))
X,y = next(iter(devloader))
X,y = next(iter(testloader))
print(X.shape,y.shape)
print(len(trainloader)+len(devloader)+len(testloader))

torch.Size([1, 501]) torch.Size([1, 1])
3139000


In [8]:
y

tensor([[4.7109, 4.9297, 4.5977, 4.3789, 4.5391, 4.6523, 4.6211, 4.7344, 4.7500,
         4.6367, 4.6250, 4.6602, 4.7305, 4.8594, 4.8047, 4.7031, 4.6758, 4.6875,
         4.6758, 4.5117, 4.5117, 4.6016, 4.6250, 4.6328, 4.6484, 4.6758, 4.7188,
         4.6289, 4.5742, 4.5078, 4.4805, 4.5078, 4.5391, 4.5195, 4.5547, 4.6055,
         4.6289, 4.6250, 4.5820, 4.5977, 4.5742, 4.5859, 4.5508, 4.5547, 4.5938,
         4.5859, 4.6172, 4.6016, 4.6211, 4.5898, 4.5547, 4.5508, 4.5664, 4.5391,
         4.5273, 4.5156, 4.5117, 4.5039, 4.4531, 4.4453, 4.4531, 4.5078, 4.5938,
         4.5859, 4.5391, 4.4883, 4.4531, 4.4570, 4.4766, 4.4883, 4.5273, 4.5742,
         4.6055, 4.6250, 4.6133, 4.5820, 4.5625, 4.5078, 4.4883, 4.4609, 4.4531,
         4.4531, 4.4766, 4.5508, 4.5312, 4.5391, 4.5391, 4.5742, 4.5664, 4.5469,
         4.5195, 4.5312, 4.5508, 4.5547, 4.5859, 4.6133, 4.5664, 4.5391, 4.5234,
         4.4688, 4.5234, 4.5273, 4.4805, 4.4961, 4.5195, 4.5273, 4.4727, 4.4766,
         4.4727, 4.5625, 4.6